In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np

# !nbdev_build_lib

In [3]:
https://ndb.nal.usda.gov/

SyntaxError: invalid syntax (329464696.py, line 1)

In [57]:
fundation_path = data_path/'FoundationFoods'
fndds_path = data_path/'FNDDS'
support_path = data_path/'support_data'
branded_path = data_path/'branded'
full_path = data_path/'full'

In [58]:
nutrients = pd.read_csv(support_path/'nutrient.csv').sort_values('id');nutrients

,id,name,unit_name,nutrient_nbr,rank
2,1001,Solids,G,201.0,200.0
3,1002,Nitrogen,G,202.0,500.0
4,1003,Protein,G,203.0,600.0
5,1004,Total lipid (fat),G,204.0,800.0
6,1005,"Carbohydrate, by difference",G,205.0,1110.0
...,...,...,...,...,...
468,2060,Ergosta-7-enol,MG,NaN,16210.0
469,2061,"Ergosta-7,22-dienol",MG,NaN,16211.0
470,2062,"Ergosta-5,7-dienol",MG,NaN,16211.0
471,2063,Verbascose,G,NaN,2450.0


In [59]:
measures = pd.read_csv(support_path/'measure_unit.csv').sort_values('id');measures

,id,name
0,1000,cup
1,1001,tablespoon
2,1002,teaspoon
3,1003,liter
4,1004,milliliter
...,...,...
117,1117,bunch
118,1118,Tablespoons
119,1119,Banana
120,1120,Onion


In [60]:
fndds = pd.read_csv(fndds_path/'food_update_log_entry.csv').sort_values('id')
fndds_nutrients = pd.read_csv(fndds_path/'food_nutrient.csv').sort_values('id')
fndd_portions = pd.read_csv(fndds_path/'food_portion.csv').sort_values('id')
attributes = pd.read_csv(fndds_path/'food_attribute.csv')


In [61]:
fndds_nutrients = (
    fndds[['id','description']].set_index('id').join(fndds_nutrients[['fdc_id','nutrient_id','amount']].set_index('fdc_id'))
    .reset_index()
    .set_index('nutrient_id')
    .join(nutrients.set_index('id'))
    .rename(columns = {'index':'id'})

)
fndds_nutrients = fndds_nutrients.pivot(index = ['id','description'],columns = ['name'],values = ['amount'])
fndds_nutrients.columns = fndds_nutrients.columns.get_level_values(1)
fndds_nutrients.columns = fndds_nutrients.columns.str.lower()


fndds_nutrients = fndds_nutrients[['energy',
                                    'protein',
                                    'carbohydrate, by difference',
                                    'total lipid (fat)']]
fndds_nutrients = fndds_nutrients.rename(columns = {'carbohydrate, by difference':'carb','total lipid (fat)':'fat'}).reset_index()

In [62]:
attributes = attributes[attributes['name'] == 'WWEIA Category description'][['fdc_id','value']].rename(columns = {'fdc_id':'id','value':'category'})

In [64]:
fndds_nutrients = fndds_nutrients.merge(attributes,on='id')

In [68]:
fndds_nutrients['category']+' '+fndds_nutrients['description']

0                                  Human milk Milk, human
1                             Milk, reduced fat Milk, NFS
2                                 Milk, whole Milk, whole
3                     Milk, whole Milk, low sodium, whole
4              Milk, whole Milk, calcium fortified, whole
                              ...                        
7078    Not included in a food category Tomatoes as in...
7079    Not included in a food category Other vegetabl...
7080    Not included in a food category Vegetables as ...
7081    Not included in a food category Sauce as ingre...
7082    Not included in a food category Industrial oil...
Length: 7083, dtype: object

In [70]:
fndds_nutrients['category'].unique()

array(['Human milk', 'Milk, reduced fat', 'Milk, whole', 'Milk, lowfat',
       'Milk, nonfat', 'Flavored milk, whole', 'Milk substitutes',
       'Yogurt, regular', 'Yogurt, Greek', 'Dips, gravies, other sauces',
       'Ice cream and frozen dairy desserts', 'Baby food: yogurt',
       'Flavored milk, nonfat', 'Flavored milk, reduced fat',
       'Flavored milk, lowfat', 'Milk shakes and other dairy drinks',
       'Smoothies and grain drinks', 'Formula, ready-to-feed',
       'Formula, prepared from powder',
       'Formula, prepared from concentrate',
       'Not included in a food category', 'Cream and cream substitutes',
       'Cream cheese, sour cream, whipped cream', 'Pudding',
       'Cakes and pies', 'Baby food: snacks and sweets', 'Cheese',
       'Cottage/ricotta cheese', 'Pizza', 'Eggs and omelets',
       'Cheese sandwiches (single code)', 'Coleslaw, non-lettuce salads',
       'Soups', 'Beef, excludes ground', 'Baby food: meat and dinners',
       'Ground beef', 'Cold cu

In [67]:
fndds_nutrients.to_sql('foods',engine,index=False,if_exists='append')


,fdc_id,value
1,1097510,Human milk
3,1097511,"Milk, reduced fat"
7,1097512,"Milk, whole"
9,1097513,"Milk, whole"
11,1097514,"Milk, whole"
...,...,...
28864,1104588,Not included in a food category
28866,1104589,Not included in a food category
28868,1104590,Not included in a food category
28870,1104591,Not included in a food category


In [20]:
attributes.pivot(columns = ['name'],values=['value'],index = ['fdc_id'])

ValueError: Index contains duplicate entries, cannot reshape

In [16]:
attributes.drop_duplicates(['fdc_id','name','value'])

,fdc_id,name,value
0,1097510,WWEIA Category number,9602
1,1097510,WWEIA Category description,Human milk
2,1097511,WWEIA Category number,1004
3,1097511,WWEIA Category description,"Milk, reduced fat"
4,1097512,NaN,leche fresca
...,...,...,...
28868,1104590,WWEIA Category description,Not included in a food category
28869,1104591,WWEIA Category number,9999
28870,1104591,WWEIA Category description,Not included in a food category
28871,1104592,WWEIA Category number,9999


In [17]:
attributes

,fdc_id,name,value
0,1097510,WWEIA Category number,9602
1,1097510,WWEIA Category description,Human milk
2,1097511,WWEIA Category number,1004
3,1097511,WWEIA Category description,"Milk, reduced fat"
4,1097512,NaN,leche fresca
...,...,...,...
28868,1104590,WWEIA Category description,Not included in a food category
28869,1104591,WWEIA Category number,9999
28870,1104591,WWEIA Category description,Not included in a food category
28871,1104592,WWEIA Category number,9999


In [191]:
fndd_portions[['fdc_id','portion_description','gram_weight']]

,fdc_id,portion_description,gram_weight
29550,1104133,"1 cup, Chiclets",191.0
1542,1098004,"1 cup, NFS",113.0
1552,1098006,"1 cup, NFS",144.0
1562,1098007,"1 cup, NFS",113.0
1572,1098008,"1 cup, NFS",113.0
...,...,...,...
131,1097536,"Quantity not specified, if in beverage",16.0
137,1097537,"Quantity not specified, if in beverage",16.0
144,1097538,"Quantity not specified, if in beverage",16.0
148,1097539,"Quantity not specified, if in beverage",16.0


In [190]:
r_fndds_portions

,index,portion_description,gram_weight
0,1097510,1 cup,246.0
1,1097510,1 fl oz,30.8
2,1097510,Quantity not specified,0.0
3,1097511,1 cup,244.0
4,1097511,1 fl oz,30.5
...,...,...,...
31761,1104566,Quantity not specified,120.0
31762,1104567,1 fl oz (no ice),31.0
31763,1104567,1 fl oz (NFS),31.0
31764,1104567,1 fl oz (with ice),23.0


branded

In [104]:
food = pd.read_csv(branded_path/'food.csv')

In [105]:
branded_food = pd.read_csv(branded_path/'branded_food.csv')

/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (2,3,4,6,9,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [112]:
food = food[['description','fdc_id']].set_index('fdc_id')

In [110]:
branded_food = branded_food[['brand_name','ingredients','fdc_id','serving_size','serving_size_unit','package_weight']].set_index('fdc_id')

In [114]:
branded_food = food.join(branded_food)

In [115]:
branded_food

,description,brand_name,ingredients,serving_size,serving_size_unit,package_weight
fdc_id,,,,,,
1105904,WESSON Vegetable Oil 1 GAL,NaN,Vegetable Oil,15.0,ml,NaN
1105905,SWANSON BROTH BEEF,NaN,"INGREDIENTS: BEEF STOCK, CONTAINS LESS THAN 2%...",240.0,ml,NaN
1105906,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,NaN,"INGREDIENTS: CLAM STOCK, POTATOES, CLAMS, CREA...",440.0,g,NaN
1105907,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,NaN,"INGREDIENTS: WATER, CREAM, BROCCOLI, CELERY, V...",440.0,g,NaN
1105908,SWANSON BROTH CHICKEN,NaN,"INGREDIENTS: CHICKEN STOCK, CONTAINS LESS THAN...",240.0,ml,NaN
...,...,...,...,...,...,...
2183200,Manuka Hunters Bush Honey 250g,Manuka Hunters,Bush Honey,10.0,g,250g
2183201,Manuka Hunters Bush Honey 500g,Manuka Hunters,Bush Honey,10.0,g,500g
2183202,Manuka Hunters Bush Honey 1kg,Manuka Hunters,Bush Honey,10.0,g,1kg


In [117]:
nutrients = pd.read_csv(branded_path/'food_nutrient.csv')

In [118]:
nutrients

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acquired
0,13706913,1105904,203,0.00,NaN,71.0,NaN,NaN,NaN,NaN,NaN
1,13706914,1105904,204,93.33,NaN,71.0,NaN,NaN,NaN,NaN,NaN
2,13706915,1105904,205,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN
3,13706924,1105904,324,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN
4,13706918,1105904,291,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
19807009,26849607,2183204,291,3.20,NaN,71.0,NaN,NaN,NaN,NaN,NaN
19807010,26849609,2183204,606,3.80,NaN,71.0,NaN,NaN,NaN,NaN,NaN
19807011,26849602,2183204,203,1.20,NaN,71.0,NaN,NaN,NaN,NaN,NaN
19807012,26849606,2183204,269,0.20,NaN,71.0,NaN,NaN,NaN,NaN,NaN
